In [30]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.grid_search import GridSearchCV
from scipy.sparse import csr_matrix, hstack
from scipy.stats import iqr
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from xgboost import XGBRegressor

%matplotlib inline

This is the final part of the capstone project where we build a simple stacker that combines the best XGBoost and MLP models.

The main idea for stacking was derived from here: https://www.kaggle.com/general/18793


### Methodology

The methodlogy for stacking is as follows: 

- **Splitting the dataset**: Split the train dataset into k folds
- **out-of-fold predictions**: Fit first stage models(Level 0 or L0) on k-1 folds and predict the kth fold
- ** full set prediction**: Fit first stage models on the whole train set and get the predictions for the test dataset. The predictions from each model  are combined into a test set where each feature is respective L0 model predicitons 
- ** stacker model**: Using the predictions from the out-of-fold set and labels of the training set , fit a stacker model (L1 model). Finally, the stacker model is used to predict test set labels based on the on the combined testset of L0 model.



### 1. Splitting of dataset

The dataset for XGBoost and MLP model is prepared by following same steps follwed in the respective model implementation stage.

- XGBoost Model: Label encoding the categorical features
- MLP Model: One-hot encoding for the binary features and label encoding for other cat variables

Also, the test set for L1 and L0 models is prepared by splitting the training set into train and test sets as we don't want to touch the actual Kaggle test set.

In [2]:
# XGBOOST model

train_xgb = pd.read_csv("input/train_fe.csv")
test_xgb = pd.read_csv('input/test_fe.csv')

final_test_xgb_id_test = test_xgb['id']

#Preparing test data
for col in test_xgb.columns:
    if test_xgb[col].dtype == 'object':
        
        lbl.fit(test_xgb[col].values)
        test_xgb[col] = lbl.transform(test_xgb[col].values)
   # else:
        #test_xgb[col] = test_xgb[col]
            
        
final_test_xgb_X = test_xgb.drop(['id', 'timestamp'], axis=1)

#Show the results 
print("Test data set has {} samples".format(final_test_xgb_X.shape))


# Data preprocessing and creating of test-train split of Train DATA
for col in train_xgb.columns:
    if train_xgb[col].dtype == 'object':
        lbl.fit(train_xgb[col].values)
        train_xgb[col] = lbl.transform(train_xgb[col].values)
        
# Training data test-train set preparation 
train_xgb_y = train_xgb['price_doc']
train_xgb_X = train_xgb.drop(['id', 'timestamp','price_doc'],axis=1)

train_xgb_y = np.array(train_xgb_y)
train_xgb_X = np.array(train_xgb_X)
# test-train splitting
xgb_train_X, xgb_test_X, xgb_train_y, xgb_test_y = train_test_split(train_xgb_X, train_xgb_y, test_size=0.3, random_state=0)
#Show the results
print("Train data set has {} features & {} labels".format(xgb_train_X.shape,xgb_train_y.shape ))   
print("Test data set has {} features & {} labels".format(xgb_test_X.shape,xgb_test_y.shape ))   

Test data set has (7662, 208) samples
Train data set has (21083, 208) features & (21083,) labels
Test data set has (9036, 208) features & (9036,) labels


In [3]:
# NN model

# importing train_mlp and test_mlp datasets
train_mlp=pd.read_csv("input/train_fe.csv")
test_mlp=pd.read_csv("input/test_fe.csv")

In [4]:
cols = train_mlp.columns
train_mlp.shape, test_mlp.shape

# marking train_mlp and test_mlp sets
n=train_mlp.shape[0]
train_mlp['data_set']=1
test_mlp['data_set']=0
test_mlp.price_doc=np.nan

final_test_mlp_id_test=test_mlp['id']

#logarithmic transformation of target did not better results
train_mlp.price_doc=(train_mlp.price_doc)
target_mlp=train_mlp.price_doc
train_mlp=train_mlp.append(test_mlp)
train_mlp.drop(['id'],axis=1,inplace=True)

# listing down binary features
binary=[]
for i in train_mlp:
    if train_mlp[i].dtypes=='object':
        #print(train_mlp[i].value_counts())
        if train_mlp[i].value_counts().shape[0]==2:
            binary.append(i)

# converting categorical varibales and handling missing data 
for i in binary:
    train_mlp[i]=pd.factorize(train_mlp[i])[0]
'''train_mlp.loc[train_mlp['ecology']=='no data','ecology_dat']=0
train_mlp.loc[train_mlp['ecology']!='no data','ecology_dat']=1
train_mlp.loc[train_mlp['ecology']=='no data','ecology']=np.nan
train_mlp.loc[train_mlp['ecology']=='poor','ecology']=1
train_mlp.loc[train_mlp['ecology']=='satisfactory','ecology']=2
train_mlp.loc[train_mlp['ecology']=='good','ecology']=3
train_mlp.loc[train_mlp['ecology']=='excellent','ecology']=4
train_mlp.ecology=pd.to_numeric(train_mlp.ecology)'''
train_mlp=pd.concat([train_mlp,pd.get_dummies(train_mlp.sub_area)],axis=1)

a=train_mlp.describe()
for i in a:
    train_mlp[i]=train_mlp[i].fillna((a.loc['min',i]-a.loc['max',i]*2))
    
train_mlp.drop(['timestamp','sub_area'],inplace=True,axis=1)

#from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols=train_mlp.columns.tolist()

train_mlp = pd.DataFrame(scaler.fit_transform(train_mlp), columns=cols)

# separating train_mlp and test_mlp after preprocessing

test_mlp=train_mlp[train_mlp['data_set']==0]
train_mlp=train_mlp[train_mlp['data_set']==1]

test_mlp.drop(['data_set','price_doc',],inplace=True,axis=1)
train_mlp.drop(['data_set','price_doc'],inplace=True,axis=1)

final_test_mlp_X = test_mlp
print("Test data set has {} samples".format(final_test_mlp_X.shape))

train_mlp = np.array(train_mlp)
target_mlp = np.array(target_mlp)

# test-train splitting
mlp_train_X, mlp_test_X, mlp_train_y, mlp_test_y = train_test_split(train_mlp, target_mlp, test_size=0.3, random_state=0)
#Show the results
print("Train data set has {} features & {} labels".format(mlp_train_X.shape,mlp_train_y.shape ))   
print("Test data set has {} features & {} labels".format(mlp_test_X.shape,mlp_test_y.shape )) 

Test data set has (7662, 353) samples
Train data set has (21083, 353) features & (21083,) labels
Test data set has (9036, 353) features & (9036,) labels


### 2. Preparing the L0-models

In [5]:
reg_xgb = xgb.XGBRegressor(num_boost_round=100,
                           eta=0.1,
                           eval_metric='rmse',
                           objective='reg:linear',
                           seed=0,
                           gamma=0,
                           max_depth=8,
                           min_child_weight=6,
                           colsample_bytree=0.6,
                           subsample=0.9)


In [6]:
%%time
# XGBoost

folds = KFold(len(xgb_train_y), shuffle=False, n_folds=3)

for k, (train_index, test_index) in enumerate(folds):
    xtr = xgb_train_X[train_index]
    ytr = xgb_train_y[train_index]
    xte, yte = xgb_train_X[test_index], xgb_train_y[test_index]

    reg_xgb.fit(xtr, ytr)
    np.savetxt('ensemble/xgb_pred_fold_{}.txt'.format(k), (reg_xgb.predict(xte)))
    np.savetxt('ensemble/xgb_test_fold_{}.txt'.format(k), yte)

Wall time: 44.3 s


In [7]:
def mlp_model():
    model = Sequential()
    model.add(Dense(128, input_dim = mlp_train_X.shape[1],init='normal' ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    # Compile Model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [8]:
%%time
# MLP

folds = KFold(len(mlp_train_y), shuffle=False, n_folds=3)

for k, (train_index, test_index) in enumerate(folds):
    xtr = mlp_train_X [train_index]
    ytr = mlp_train_y[train_index]
    xte, yte = mlp_train_X[test_index], mlp_train_y[test_index]
    
    reg_mlp = mlp_model()
    fit = reg_mlp.fit(xtr, ytr, batch_size=32, epochs=100, verbose=0)
    pred = reg_mlp.predict(xte, batch_size=32)
    np.savetxt('ensemble/mlp_pred_fold_{}.txt'.format(k), pred)
    np.savetxt('ensemble/mlp_test_fold_{}.txt'.format(k), yte)

C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 15min 19s


### Training on the whole dataset

<p> We train the same models on the whole training set (xgb_train_X, mlp_train_X and corresponding labels xgb_train_y, mlp_train_y) and generate predictions for the test set (xgb_test_X, mlp_test_X). <br>It is important to note that we do have labels for the test set, but we don't allow our L0-model see them instead it uses from the one of the 3 folds from the training set .
<p/>

In [9]:
%%time
# XGBoost on whole set and predicted using test set

reg_xgb.fit(xgb_train_X, xgb_train_y)
pred_xgb = reg_xgb.predict(xgb_test_X)
np.savetxt('ensemble/xgb_pred_test.txt'.format(k), pred_xgb)

Wall time: 21.8 s


In [10]:
%%time
#MLP on whole set and predicted using test set

reg_mlp = mlp_model()
fit = reg_mlp.fit(mlp_train_X, mlp_train_y, batch_size=32, epochs=100, verbose=0)
pred_mlp = reg_mlp.predict(mlp_test_X, batch_size=32)
np.savetxt('ensemble/mlp_pred_test.txt'.format(k), pred_mlp)

C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 7min 39s


### 3. L1-model training

<p>When the previous stage is completed, we have generated out-of-fold and test set predictions, which we can now use to train the stacker.<br/>Firstly, Loading out-of-fold predictions:<p/>

In [11]:
train_xgb1 = np.loadtxt('ensemble/xgb_pred_fold_0.txt')
train_xgb2 = np.loadtxt('ensemble/xgb_pred_fold_1.txt')
train_xgb3 = np.loadtxt('ensemble/xgb_pred_fold_2.txt')

train_mlp1 = np.loadtxt('ensemble/mlp_pred_fold_0.txt')
train_mlp2 = np.loadtxt('ensemble/mlp_pred_fold_1.txt')
train_mlp3 = np.loadtxt('ensemble/mlp_pred_fold_2.txt')

#### Sanity Check

<p>Checking if the stacked out-of-fold predictions are within the relavent limits.<p/>

In [12]:
# combing the out-of-fold predictions

train_xgb_folds = np.hstack((train_xgb1,train_xgb2,train_xgb3))
train_mlp_folds = np.hstack((train_mlp1,train_mlp2,train_mlp3))

In [13]:
# MSE of XGBoost combined predictions

mean_squared_error(xgb_train_y, train_xgb_folds)**0.5

2263499.0271548009

In [14]:
# MSE of MLP combined predictions

mean_squared_error(mlp_train_y, train_mlp_folds)**0.5

2520728.1733623822

<p>Both the above predictions for the combined out-of-fold predictions have an error within the expected range.  In order to get a baseline score for the stacker, let's calculate predictions with the entire training set.
<p/>

In [15]:
test_xgb_pred = np.loadtxt('ensemble/xgb_pred_test.txt')
test_mlp_pred = np.loadtxt('ensemble/mlp_pred_test.txt')

In [16]:
mse_xgb_test = mean_squared_error(xgb_test_y, test_xgb_pred)**0.5
mse_mlp_test = mean_squared_error(mlp_test_y, test_mlp_pred)**0.5
print ("Baseline MSE which we need to improve with stacking, XGB: {}; MLP: {}.".format(mse_xgb_test, mse_mlp_test))

Baseline MSE which we need to improve with stacking, XGB: 2238255.3947151075; MLP: 2615654.310512517.


<p>The predictions from the whole training data sets a baseline score that the stacker should better
<p/>

### Training L1-model

This final step involves combining the L0-models (XGB and MLP). The selection of algorithm is very cruical, a simple regressor is preferable as it reduces the chances of overfitting. Hence, let's select linear regression model.

In [17]:
l1_train_x = np.vstack((train_xgb_folds, train_mlp_folds)).T
l1_test_x = np.vstack((test_xgb_pred, test_mlp_pred)).T
l1_train_y = mlp_train_y 
l1_test_y = mlp_test_y

In [18]:
# dimensions 
print ("Xtrain shape:", l1_train_x.shape)
print ("ytrain shape:", l1_train_y.shape)
print ("Xtest shape:", l1_test_x.shape)
print ("ytest shape:", l1_test_y.shape)

Xtrain shape: (21083, 2)
ytrain shape: (21083,)
Xtest shape: (9036, 2)
ytest shape: (9036,)


<p>We now fit a very basic linear regression and get the predictions for the final test set:</p>

In [19]:
reg = LinearRegression()

# Note that normalizing the data in case of linear models is very important
reg.fit((l1_train_x), (l1_train_y))
reg_pred = (reg.predict((l1_test_x)))

mse_stacker = mean_squared_error(l1_test_y, reg_pred)**0.5
mse_stacker

2237728.5415377445

In [20]:
print ("RMSE for XGB:", mse_xgb_test)
print ("RMSE for MLP:", mse_mlp_test)
print ("RMSE for stacker:", mse_stacker)

RMSE for XGB: 2238255.39472
RMSE for MLP: 2615654.31051
RMSE for stacker: 2237728.54154


In [21]:
reg.coef_

array([ 0.82507246,  0.21255648])

<p>The regression coefficient indicates the final prediction is a linear combination of XGB and MLP models and has following relationship:
 ### `Final_Prediction = 0.80*XGB_Pred + 0.23*MLP_Pred` 
<p>


The final ensemble model has CV error of 2236490.76654, that is best score achived compared to the individual models used in the ensemble.

## Validation

In [22]:
class Stacker(object):
    def __init__(self, xgboost_func, mlp_func, train_path='input/train_fe.csv',test_path= 'input/test_fe.csv',
                 seed=0, test_size=0.25, **kwargs):
        self.seed = seed
        self.test_size = test_size
        self.xgboost_func = xgboost_func
        self.train_path = train_path
        self.test_path = test_path
        self.mlp_func = mlp_func
        self.mlp_fit_kwargs = kwargs.get('mlp_fit_kwargs', {'nb_epoch': 30, 'batch_size': 32, 'verbose': 1})
        self.mlp_predict_kwargs = kwargs.get('mlp_predict_kwargs', {'batch_size': 32, 'verbose': 1})

    def stack_and_compare(self):
        xg_xtr, xg_xte, xg_ytr, xg_yte = self.preprocess(encoding='XGB', transform_label=False) #for XGBoost
        mlp_xtr, mlp_xte, mlp_ytr, mlp_yte = self.preprocess(encoding='MLP', transform_label=False) # for MLP

        xgb_folds = self.predict_folds(self.xgboost_func, xg_xtr, xg_ytr)
        mlp_folds = (self.predict_folds(self.mlp_func, mlp_xtr, mlp_ytr,
                                              fit_kwargs=self.mlp_fit_kwargs, predict_kwargs=self.mlp_predict_kwargs))

        xgb_pred_hold = self.predict_holdout(self.xgboost_func, xg_xtr, xg_ytr, xg_xte)
        mlp_pred_hold = (self.predict_holdout(self.mlp_func, mlp_xtr, mlp_ytr, mlp_xte,
                                                    fit_kwargs=self.mlp_fit_kwargs,
                                                    predict_kwargs=self.mlp_predict_kwargs))

        score_xgb, score_mlp = self.evaluate_estimators(xgb_pred_hold, mlp_pred_hold, xg_yte)
        print('Single model performance:', 'xgb:', score_xgb, ',', 'mlp:', score_mlp)

        stacker = self.stack(xgb_folds, mlp_folds, xg_ytr)
        score_stacker = self.evaluate_stacker(stacker, xgb_pred_hold, mlp_pred_hold, xg_yte)

        print({'xgb': score_xgb**0.5, 'mlp': score_mlp**0.5, 'stacker': score_stacker**0.5})
        return {'xgb': score_xgb**0.5, 'mlp': score_mlp**0.5, 'stacker': score_stacker**0.5}

    def preprocess(self, encoding='XGB', transform_label=False):
        train = pd.read_csv(self.train_path)
        test = pd.read_csv(self.test_path)
        if transform_label:
            train['price_doc'] = (train['price_doc'])
        cat_features = [x for x in train.select_dtypes(include=['object']).columns if x not in ['id', 'price_doc']]
        if encoding == 'XGB':
            for col in train.columns:
                if train[col].dtype == 'object':
                    lbl.fit(train[col].values)
                    train[col] = lbl.transform(train[col].values)
            #print('XGB Label',train['price_doc'].head())
            
            features = [x for x in train.columns if x not in ['id', 'price_doc']]
            train_x = np.array(train[features])
            print(train['price_doc'].head(n=2))
            train_y = np.array(train['price_doc'])
            
        elif encoding == 'MLP':
            
            cols = train.columns
            # marking train_mlp and test_mlp sets
            n = train.shape[0]
            #train['data_set'] = 1
            #test['data_set'] = 0
            #test.price_doc = np.nan

            # logarithmic transformation of target did not better results
            #train.price_doc = (train.price_doc)
            target_mlp = train.price_doc
            #train = train.append(test)
            train.drop(['id'], axis=1, inplace=True)

            # listing down binary features
            binary = []
            for i in train:
                if train[i].dtypes == 'object':
                    # print(train_mlp[i].value_counts())
                    if train[i].value_counts().shape[0] == 2:
                        binary.append(i)

            # converting categorical varibales and handling missing data
            for i in binary:
                train[i] = pd.factorize(train[i])[0]

            train = pd.concat([train, pd.get_dummies(train.sub_area)], axis=1)
            #NaN treatment
            a = train.describe()
            for i in a:
                train[i] = train[i].fillna((a.loc['min', i] - a.loc['max', i] * 2))

            train.drop(['timestamp', 'sub_area'], inplace=True, axis=1)

            # from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            cols = train.columns.tolist()

            train = pd.DataFrame(scaler.fit_transform(train), columns=cols)

            # separating train_mlp and test_mlp after preprocessing

            #test = train[train['data_set'] == 0]
            #train = train[train['data_set'] == 1]

            # test.drop(['data_set' ], inplace=True, axis=1)
            #train.drop(['data_set'], inplace=True, axis=1)
            
            final_test_mlp_X = train
            print("Train data set has {} samples".format(final_test_mlp_X.shape))
            #print('MLP Label',train['price_doc'].head())
            
            features = [x for x in train.columns if x not in ['id', 'price_doc']]
            train_x = np.array(train[features])
            print(target_mlp.head(n=2))
            train_y = np.array(target_mlp)
        else:
            raise Exception("Correct value of 'encoding' is required. Possible values of encoding=['one-hot', 'label']")
        features = [x for x in train.columns if x not in ['id', 'price_doc']]
        
        '''train_x = np.array(train[features])
        print(train['price_doc'].head(n=2))
        train_y = np.array(train['price_doc'])
        '''
        x_tr, x_te, y_tr, y_te = train_test_split(train_x, train_y, test_size=self.test_size, random_state=self.seed)
        return x_tr, x_te, y_tr, y_te

    def predict_folds(self, model_func, xtrain, ytrain, fit_kwargs={}, predict_kwargs={}):
        folds = KFold(len(ytrain), shuffle=False, n_folds=3)
        fold_preds = np.zeros(len(ytrain))
        fold_shape = fold_preds.shape
        for k, (train_index, test_index) in enumerate(folds):
            xtr = xtrain[train_index]
            ytr = ytrain[train_index]
            estimator = model_func(xtrain.shape[1])
            xte, yte = xtrain[test_index], ytrain[test_index]
            estimator.fit(xtr, ytr, **fit_kwargs)
            fold_preds[test_index] = np.reshape(estimator.predict(xte, **predict_kwargs),-1)
        return fold_preds

    def predict_holdout(self, model_func, xtrain, ytrain, xtest, fit_kwargs={}, predict_kwargs={}):
        estimator = model_func(xtrain.shape[1])
        estimator.fit(xtrain, ytrain, **fit_kwargs)
        return estimator.predict(xtest, **predict_kwargs)

    def evaluate_estimators(self, xgb_pred, mlp_pred, test_y):
        return (mean_squared_error((xgb_pred), (test_y))), (mean_squared_error((mlp_pred),(test_y)))

    def stack(self, xgb_oof, mlp_oof, oof_y):
        assert len(xgb_oof) == len(mlp_oof)
        oof_x = np.vstack((xgb_oof, mlp_oof)).T
        metaestimator = LinearRegression()
        metaestimator.fit(oof_x, oof_y)
        return metaestimator

    def evaluate_stacker(self, stacker, xgb_pred, mlp_pred, holdout_y):
        holdout_pred = np.hstack((xgb_pred.reshape(len(xgb_pred), 1), np.array(mlp_pred)))
        predictions = stacker.predict(holdout_pred)
        score = mean_squared_error((predictions), (holdout_y))
        return score

In [23]:
seeds = [10,20,30,40,50]
#seeds = [0]
final_scores = []

# Preparing XGBoost model function
xgbst = lambda _: xgb.XGBRegressor(num_boost_round=100,
                           eta=0.1,
                           eval_metric='rmse',
                           objective='reg:linear',
                           seed=0,
                           gamma=0,
                           max_depth=8,
                           min_child_weight=6,
                           colsample_bytree=0.6,
                           subsample=0.9)

# Preparing MLP model function. We need to pass input dimension into
# this function, which is done in Stacker class
def hyper_model(dim):
    model = Sequential()
    model.add(Dense(128, input_dim =dim,init='normal' ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    # Compile Model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [24]:
mlp_fit_params = {'epochs': 100, 'batch_size': 32, 'verbose': 1}
mlp_pred_params = {'batch_size': 32, 'verbose': 1}

In [25]:
%%time
%%capture cap --no-stderr 

for seed in seeds:
    stacker = Stacker(xgbst, hyper_model, seed=seed,
                  mlp_fit_kwargs=mlp_fit_params, mlp_predict_kwargs=mlp_pred_params)
    score = stacker.stack_and_compare()
    final_scores.append(score)
    print ("Seed {} completed with scores :: {}".format(seed, score))


C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
C:\Continuum\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=353, kernel_initializer="normal")`
C:\Continuum\Anaconda3\envs\tf

Wall time: 2h 8min 37s


In [26]:
with open('stacker_validation.txt', 'w') as f:
    f.write(cap.stdout)

In [27]:
for seed, score in zip(seeds, final_scores):
    print ("SEED: {}".format(seed))
    print ("SCORE: {}".format(score))

SEED: 10
SCORE: {'stacker': 2193287.6247396455, 'xgb': 2207972.477460864, 'mlp': 2466970.5167615437}
SEED: 20
SCORE: {'stacker': 2171958.4140811809, 'xgb': 2178806.3050570297, 'mlp': 2479116.8569160611}
SEED: 30
SCORE: {'stacker': 2255960.6154502756, 'xgb': 2255339.2048879955, 'mlp': 2594524.342251874}
SEED: 40
SCORE: {'stacker': 2196563.2986961356, 'xgb': 2202009.8203167059, 'mlp': 2517315.4996815301}
SEED: 50
SCORE: {'stacker': 2236356.0243481873, 'xgb': 2246863.4458257603, 'mlp': 2576243.6124122702}


In [28]:
scores_df = pd.DataFrame(final_scores)

In [29]:
scores_df.describe()

,mlp,stacker,xgb
count,5.000000e+00,5.000000e+00,5.000000e+00
mean,2.526834e+06,2.210825e+06,2.218198e+06
std,5.695274e+04,3.431699e+04,3.209167e+04
min,2.466971e+06,2.171958e+06,2.178806e+06
25%,2.479117e+06,2.193288e+06,2.202010e+06
50%,2.517315e+06,2.196563e+06,2.207972e+06
75%,2.576244e+06,2.236356e+06,2.246863e+06
max,2.594524e+06,2.255961e+06,2.255339e+06


## Final Predictions for Submission

In [22]:
final_test_mlp_X.shape, final_test_xgb_X.shape

((7662, 353), (7662, 208))

In [23]:
# xgb predictions
final_xgb_pred_y = reg_xgb.predict(np.array(final_test_xgb_X))
final_xgb_pred_y.shape

(7662,)

In [24]:
# mlp predictions

final_mlp_pred_y=reg_mlp.predict(final_test_mlp_X.as_matrix(), batch_size=32, verbose=0)
final_mlp_pred_y_rs = np.reshape(final_mlp_pred_y, final_mlp_pred_y.shape[0])
final_mlp_pred_y.shape, final_mlp_pred_y_rs.shape

((7662, 1), (7662,))

In [25]:
A = 0.80887096
B = 0.23771801

final_stacker_pred_y = A*final_xgb_pred_y +  B* final_mlp_pred_y_rs

In [26]:
subs=pd.DataFrame({'id':final_test_mlp_id_test.as_matrix(),'price_doc':final_stacker_pred_y})
subs.head()
subs.to_csv("output/final_stacker_pred_20170819.csv",index=False)

The final submission gave score of 0.32622, this the best score achieved compared to all the indidual models.